In [1]:
#Importing Dependencies
import os             # use it to manipulate file/folder directories
import shutil         # to copy or move any data from one dir to another
import numpy as np    # to convert image data to array format
import pandas as pd   # use it to make dataframes
from PIL import Image # use it to access image file in a directory and check its type

In [2]:
# Set up retrieval of file info, img label, src and dest dir's
src_img_dir = "./jpeg"
dest_parent_dir = "./calc_train"
csv_file_path = "./csv/calc_case_description_train_set.csv"
df = pd.read_csv(csv_file_path)
df['folder_path'] = df['image file path'].str.split('/').str[2]
# Drop unnecessary columns
df = df.drop(['image file path', 'cropped image file path', 'ROI mask file path'], axis=1)
df.head()

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,folder_path
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,1.3.6.1.4.1.9590.100.1.2.474143160103683865197...
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,1.3.6.1.4.1.9590.100.1.2.250596608311207922527...
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,1.3.6.1.4.1.9590.100.1.2.228699627313487111012...
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,1.3.6.1.4.1.9590.100.1.2.104743410411133110629...
4,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN_WITHOUT_CALLBACK,3,1.3.6.1.4.1.9590.100.1.2.406725628213826290127...


In [3]:
# Access 'pathology' and 'folder_path' columns
selected_data_df = df[['pathology', 'folder_path']]
selected_data_df.head()

,pathology,folder_path
0,MALIGNANT,1.3.6.1.4.1.9590.100.1.2.474143160103683865197...
1,MALIGNANT,1.3.6.1.4.1.9590.100.1.2.250596608311207922527...
2,BENIGN,1.3.6.1.4.1.9590.100.1.2.228699627313487111012...
3,BENIGN,1.3.6.1.4.1.9590.100.1.2.104743410411133110629...
4,BENIGN_WITHOUT_CALLBACK,1.3.6.1.4.1.9590.100.1.2.406725628213826290127...


In [4]:
# Iterate over the rows and access values:
def is_valid_image(file_path):
    """Return True if the file is a valid image."""
    try:
        with Image.open(file_path) as img:
            img.verify()  # verify that it is, in fact, an image
        return True
    except (IOError, SyntaxError):
        return False

    
def image_type(image_path):
    with Image.open(image_path) as img:
        # Convert image to numpy array
        data = np.array(img)
        # Check if RGB image
        if len(data.shape) == 3:
            # Check if all channels have same values which indicates grayscale
            if np.array_equal(data[:,:,0], data[:,:,1]) and np.array_equal(data[:,:,1], data[:,:,2]):
                unique_values = np.unique(data[:,:,0])
                # Check if only two unique values (0 and 255) which indicates binary
                if len(unique_values) == 2 and 0 in unique_values and 255 in unique_values:
                    return "Binary"
                else:
                    return "Grayscale"
            else:
                return "Color"

        # Check if single channel image
        elif len(data.shape) == 2:
            unique_values = np.unique(data)
            # Check if only two unique values (0 and 255) which indicates binary
            if len(unique_values) == 2 and 0 in unique_values and 255 in unique_values:
                return "Binary"
            else:
                return "Grayscale"

# Iteration routine to copy and transfer images from source to the labeled directories
dest_folder = "" # destination folder
for index, row in selected_data_df.iterrows():
    pathology = row['pathology']
    folder_path = row['folder_path']
    print(folder_path)    
    dest_folder = dest_parent_dir + "/" + pathology    
    # Iterate through each individual file in a folder
    target_dir = src_img_dir + "/" + folder_path
    contents = os.listdir(target_dir)
    for item in contents:
        full_path_toFile = os.path.join(target_dir, item)
        if is_valid_image(full_path_toFile):
            img_type = image_type(full_path_toFile)
            if img_type:
                dest_file = os.path.join(dest_folder, item)
                shutil.copy2(full_path_toFile, dest_file)                
                print(f"Image File ({img_type}): {item}")
            else:
                print(f"Not an Image File: {item}")
        else:
            print(f"Not an Image File: {item}")

1.3.6.1.4.1.9590.100.1.2.47414316010368386519740343172775938548
Image File (Grayscale): 1-188.jpg
1.3.6.1.4.1.9590.100.1.2.250596608311207922527805669693579696727
Image File (Grayscale): 1-189.jpg
1.3.6.1.4.1.9590.100.1.2.228699627313487111012474405462022068297
Image File (Grayscale): 1-190.jpg
1.3.6.1.4.1.9590.100.1.2.104743410411133110629448544090466900446
Image File (Grayscale): 1-191.jpg
1.3.6.1.4.1.9590.100.1.2.406725628213826290127343763811145520834
Image File (Grayscale): 1-192.jpg
1.3.6.1.4.1.9590.100.1.2.406725628213826290127343763811145520834
Image File (Grayscale): 1-192.jpg
1.3.6.1.4.1.9590.100.1.2.406725628213826290127343763811145520834
Image File (Grayscale): 1-192.jpg
1.3.6.1.4.1.9590.100.1.2.14784689311239808604822345534163797907
Image File (Grayscale): 1-193.jpg
1.3.6.1.4.1.9590.100.1.2.14784689311239808604822345534163797907
Image File (Grayscale): 1-193.jpg
1.3.6.1.4.1.9590.100.1.2.14784689311239808604822345534163797907
Image File (Grayscale): 1-193.jpg
1.3.6.1.4.1.95

Image File (Grayscale): 1-260.jpg
1.3.6.1.4.1.9590.100.1.2.33861711511809892034543074661890550667
Image File (Grayscale): 1-261.jpg
1.3.6.1.4.1.9590.100.1.2.33861711511809892034543074661890550667
Image File (Grayscale): 1-261.jpg
1.3.6.1.4.1.9590.100.1.2.224225944413287957229090139042821529119
Image File (Grayscale): 1-262.jpg
1.3.6.1.4.1.9590.100.1.2.318543965711217763213591571541538853788
Image File (Grayscale): 1-263.jpg
1.3.6.1.4.1.9590.100.1.2.318543965711217763213591571541538853788
Image File (Grayscale): 1-263.jpg
1.3.6.1.4.1.9590.100.1.2.387302091512953100939990337603904295050
Image File (Grayscale): 1-264.jpg
1.3.6.1.4.1.9590.100.1.2.325202908012267823903213633892192510338
Image File (Grayscale): 1-265.jpg
1.3.6.1.4.1.9590.100.1.2.8433705612119628039709943644165314222
Image File (Grayscale): 1-268.jpg
1.3.6.1.4.1.9590.100.1.2.173470568512513530609795736530062754770
Image File (Grayscale): 1-269.jpg
1.3.6.1.4.1.9590.100.1.2.323217412710482457939027237711644777753
Image File (Gr

Image File (Grayscale): 1-037.jpg
1.3.6.1.4.1.9590.100.1.2.166241369613226623713666231473435390272
Image File (Grayscale): 1-039.jpg
1.3.6.1.4.1.9590.100.1.2.103501940211629107411815722493638210833
Image File (Grayscale): 1-043.jpg
1.3.6.1.4.1.9590.100.1.2.306434300513050292904954528021879535685
Image File (Grayscale): 1-044.jpg
1.3.6.1.4.1.9590.100.1.2.76505703411657502419535945180203364480
Image File (Grayscale): 1-045.jpg
1.3.6.1.4.1.9590.100.1.2.155630705013735500834345117482735990480
Image File (Grayscale): 1-050.jpg
1.3.6.1.4.1.9590.100.1.2.276742865512373902711968137433743642693
Image File (Grayscale): 1-051.jpg
1.3.6.1.4.1.9590.100.1.2.287957643811383258429935679350984600309
Image File (Grayscale): 1-052.jpg
1.3.6.1.4.1.9590.100.1.2.368310316213755494128599891293302308315
Image File (Grayscale): 1-053.jpg
1.3.6.1.4.1.9590.100.1.2.202764963511848978636750485820493638315
Image File (Grayscale): 1-054.jpg
1.3.6.1.4.1.9590.100.1.2.77825672913766419121190299220745993179
Image File (

Image File (Grayscale): 1-144.jpg
1.3.6.1.4.1.9590.100.1.2.212959337713547204430565679633073241823
Image File (Grayscale): 1-145.jpg
1.3.6.1.4.1.9590.100.1.2.241738864711213044712467096672933890972
Image File (Grayscale): 1-146.jpg
1.3.6.1.4.1.9590.100.1.2.234243541612728769418429395573518420644
Image File (Grayscale): 1-147.jpg
1.3.6.1.4.1.9590.100.1.2.164781356010870637214045014043866075278
Image File (Grayscale): 1-148.jpg
1.3.6.1.4.1.9590.100.1.2.164781356010870637214045014043866075278
Image File (Grayscale): 1-148.jpg
1.3.6.1.4.1.9590.100.1.2.195203275012771218213044732193030045372
Image File (Grayscale): 1-149.jpg
1.3.6.1.4.1.9590.100.1.2.115982257811514363324326263192430686573
Image File (Grayscale): 1-150.jpg
1.3.6.1.4.1.9590.100.1.2.176216452612060895930870780631833050383
Image File (Grayscale): 1-151.jpg
1.3.6.1.4.1.9590.100.1.2.35124586712113301126720206281638387089
Image File (Grayscale): 1-152.jpg
1.3.6.1.4.1.9590.100.1.2.34446056412965364400659373073901888276
Image File (

Image File (Grayscale): 1-212.jpg
1.3.6.1.4.1.9590.100.1.2.286659962612608383206161076632186976247
Image File (Grayscale): 1-213.jpg
1.3.6.1.4.1.9590.100.1.2.114424168911311052813833481510260041688
Image File (Grayscale): 1-216.jpg
1.3.6.1.4.1.9590.100.1.2.114424168911311052813833481510260041688
Image File (Grayscale): 1-216.jpg
1.3.6.1.4.1.9590.100.1.2.114424168911311052813833481510260041688
Image File (Grayscale): 1-216.jpg
1.3.6.1.4.1.9590.100.1.2.309752174610992973034364663291781979620
Image File (Grayscale): 1-217.jpg
1.3.6.1.4.1.9590.100.1.2.309752174610992973034364663291781979620
Image File (Grayscale): 1-217.jpg
1.3.6.1.4.1.9590.100.1.2.309752174610992973034364663291781979620
Image File (Grayscale): 1-217.jpg
1.3.6.1.4.1.9590.100.1.2.248911735611581654701523977480859117492
Image File (Grayscale): 1-218.jpg
1.3.6.1.4.1.9590.100.1.2.152922289013154069005983046780699449457
Image File (Grayscale): 1-219.jpg
1.3.6.1.4.1.9590.100.1.2.179094779913865355328314942890032167835
Image File

...

Image File (Grayscale): 1-081.jpg
1.3.6.1.4.1.9590.100.1.2.107208772712756184727248595500844018939
Image File (Grayscale): 1-082.jpg
1.3.6.1.4.1.9590.100.1.2.107208772712756184727248595500844018939
Image File (Grayscale): 1-082.jpg
1.3.6.1.4.1.9590.100.1.2.69593508413111090311950590930695136313
Image File (Grayscale): 1-083.jpg
1.3.6.1.4.1.9590.100.1.2.352822755513452795201836990730140870919
Image File (Grayscale): 1-084.jpg
1.3.6.1.4.1.9590.100.1.2.422123520011922899217541945691484632474
Image File (Grayscale): 1-085.jpg
1.3.6.1.4.1.9590.100.1.2.96248553913743164001617432932817446188
Image File (Grayscale): 1-086.jpg
1.3.6.1.4.1.9590.100.1.2.48108565012803469540186438433421578888
Image File (Grayscale): 1-087.jpg
1.3.6.1.4.1.9590.100.1.2.104098193313490577933291262873930417517
Image File (Grayscale): 1-088.jpg
1.3.6.1.4.1.9590.100.1.2.74414112111016019039766101982433139856
Image File (Grayscale): 1-089.jpg
1.3.6.1.4.1.9590.100.1.2.423283577712821737707939593061526028130
Image File (Gr

Image File (Grayscale): 1-166.jpg
1.3.6.1.4.1.9590.100.1.2.380327564311857678400848846711545959410
Image File (Grayscale): 1-167.jpg
1.3.6.1.4.1.9590.100.1.2.303116768511829478820680925500312061481
Image File (Grayscale): 1-171.jpg
1.3.6.1.4.1.9590.100.1.2.142907773911947688702213113893866359104
Image File (Grayscale): 1-172.jpg
1.3.6.1.4.1.9590.100.1.2.296655292410858880841776257620436965580
Image File (Grayscale): 1-173.jpg
1.3.6.1.4.1.9590.100.1.2.327546655412429907703257655470092494966
Image File (Grayscale): 1-174.jpg
1.3.6.1.4.1.9590.100.1.2.5560655811630664812485794823416544257
Image File (Grayscale): 1-175.jpg
1.3.6.1.4.1.9590.100.1.2.321370481911159457023509364911008063546
Image File (Grayscale): 1-178.jpg
1.3.6.1.4.1.9590.100.1.2.125646349412437530720906699543571013327
Image File (Grayscale): 1-179.jpg
1.3.6.1.4.1.9590.100.1.2.56085006113084566424056551783284458998
Image File (Grayscale): 1-182.jpg
1.3.6.1.4.1.9590.100.1.2.26524748811152512242348842641634506174
Image File (Gr

Image File (Grayscale): 1-259.jpg
1.3.6.1.4.1.9590.100.1.2.228757952612837468335714490132465370462
Image File (Grayscale): 1-259.jpg
1.3.6.1.4.1.9590.100.1.2.413602028113120193338228195140084272353
Image File (Grayscale): 1-260.jpg
1.3.6.1.4.1.9590.100.1.2.413602028113120193338228195140084272353
Image File (Grayscale): 1-260.jpg
1.3.6.1.4.1.9590.100.1.2.148865302612041838439623934924266740496
Image File (Grayscale): 1-261.jpg
1.3.6.1.4.1.9590.100.1.2.139947696511163671102138561880159238288
Image File (Grayscale): 1-262.jpg
1.3.6.1.4.1.9590.100.1.2.212425941512870558719149152530869893055
Image File (Grayscale): 1-263.jpg
1.3.6.1.4.1.9590.100.1.2.339745344212025184242719239031668459005
Image File (Grayscale): 1-264.jpg
1.3.6.1.4.1.9590.100.1.2.320804262613185661903034489902983101730
Image File (Grayscale): 1-265.jpg
1.3.6.1.4.1.9590.100.1.2.80924197111890944022045879071454363046
Image File (Grayscale): 1-266.jpg
1.3.6.1.4.1.9590.100.1.2.46787213813776042820460758302407385413
Image File (

Image File (Grayscale): 1-018.jpg
1.3.6.1.4.1.9590.100.1.2.129791061511313973927878918694190000311
Image File (Grayscale): 1-018.jpg
1.3.6.1.4.1.9590.100.1.2.181841823211422221335388532932156762334
Image File (Grayscale): 1-019.jpg
1.3.6.1.4.1.9590.100.1.2.181841823211422221335388532932156762334
Image File (Grayscale): 1-019.jpg
1.3.6.1.4.1.9590.100.1.2.181841823211422221335388532932156762334
Image File (Grayscale): 1-019.jpg
1.3.6.1.4.1.9590.100.1.2.114030169011500101820138787040137395797
Image File (Grayscale): 1-020.jpg
1.3.6.1.4.1.9590.100.1.2.114030169011500101820138787040137395797
Image File (Grayscale): 1-020.jpg
1.3.6.1.4.1.9590.100.1.2.380585108912835672335427219224208339965
Image File (Grayscale): 1-021.jpg
1.3.6.1.4.1.9590.100.1.2.299740424912592591728594490160937069544
Image File (Grayscale): 1-022.jpg
1.3.6.1.4.1.9590.100.1.2.213607608912884835305932847630555559538
Image File (Grayscale): 1-023.jpg
1.3.6.1.4.1.9590.100.1.2.238085912110799023135128785212540901367
Image File

Image File (Grayscale): 1-093.jpg
1.3.6.1.4.1.9590.100.1.2.74005715112183196613423200034069472130
Image File (Grayscale): 1-094.jpg
1.3.6.1.4.1.9590.100.1.2.235425023411867664608986305432637770251
Image File (Grayscale): 1-095.jpg
1.3.6.1.4.1.9590.100.1.2.337050783411140427214197121660923286256
Image File (Grayscale): 1-096.jpg
1.3.6.1.4.1.9590.100.1.2.351418242812953841607082290431510352119
Image File (Grayscale): 1-097.jpg
1.3.6.1.4.1.9590.100.1.2.412648341012113205720900809264073133781
Image File (Grayscale): 1-098.jpg
1.3.6.1.4.1.9590.100.1.2.141617340212085510809039998740404392474
Image File (Grayscale): 1-099.jpg
1.3.6.1.4.1.9590.100.1.2.333575065312722552713541787451023988690
Image File (Grayscale): 1-100.jpg
1.3.6.1.4.1.9590.100.1.2.370030936310415648030051424362413291051
Image File (Grayscale): 1-101.jpg
1.3.6.1.4.1.9590.100.1.2.206211701612209078000811887921489167359
Image File (Grayscale): 1-102.jpg
1.3.6.1.4.1.9590.100.1.2.209140269413286775600519118310137054510
Image File 

In [15]:
print(img_dir + "/") #+ folder_path

./jpeg/
